## Appendix: Optimizing data filling via Vectorization 

* What is vectorizing/vectorization? 
  
* When can we vectorize/under what conditions? (Order-dependent or independent operations)
  
* In Python, what tools do we use to vectorize?
   
* How do we vectorize? *Coding demonstration* 
  
* Enjoy making your code run faster? Stay posted for the SSDS MOPS Workshop (**M**anaging & **O**ptimizing **P**erformance and **S**calability - and Memory!)